In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [2]:

# Directories for training and validation images
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'


In [3]:

# Function to create a dataframe with image paths and labels
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels


In [4]:

# Creating train dataframe
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

# Creating test dataframe
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)
print(test['image'])

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                    image     label
0       images/validation\angry\10052.jpg     angry
1       images/validation\angry\10065.jpg     angry
2       im

In [5]:

# Function to extract features (loading and reshaping images)
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features


In [6]:

# Extracting features for train and test datasets
train_features = extract_features(train['image']) 
test_features = extract_features(test['image'])

# Normalizing the image data
x_train = train_features / 255.0
x_test = test_features / 255.0


100%|██████████| 7066/7066 [00:37<00:00, 189.82it/s]


In [7]:

# Encoding the labels
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

# One-hot encoding of labels
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [8]:

# Creating the model
model = Sequential()
# Adding convolutional and pooling layers with Batch Normalization and Dropout
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Flattening the output from convolutional layers
model.add(Flatten())
# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# Output layer for classification into 7 classes
model.add(Dense(7, activation='softmax'))


w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [11]:

# Fitting the model with augmented data
datagen.fit(x_train)
model.fit(datagen.flow(x_train, y_train, batch_size=128), epochs=100, validation_data=(x_test, y_test))


Epoch 1/100


w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


226/226 ━━━━━━━━━━━━━━━━━━━━ 429s 2s/step - accuracy: 0.2060 - loss: 2.1136 - val_accuracy: 0.2583 - val_loss: 1.8343
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 458s 2s/step - accuracy: 0.2440 - loss: 1.8062 - val_accuracy: 0.2583 - val_loss: 1.8310
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 466s 2s/step - accuracy: 0.2520 - loss: 1.7884 - val_accuracy: 0.2938 - val_loss: 1.7240
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.2701 - loss: 1.7620 - val_accuracy: 0.2363 - val_loss: 1.8574
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 360s 2s/step - accuracy: 0.2991 - loss: 1.7210 - val_accuracy: 0.3368 - val_loss: 1.6519
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.3229 - loss: 1.6872 - val_accuracy: 0.3221 - val_loss: 1.6689
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 336s 1s/step - accuracy: 0.3503 - loss: 1.6333 - val_accuracy: 0.3624 - val_loss: 1.6405
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 278s 1s/step - accuracy: 0.3680 - loss: 1.6039 - val_accuracy

In [12]:

# Saving the model architecture and weights
model_json = model.to_json()
with open("emotiondetector3.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector3.h5")


In [13]:

# Loading the saved model
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")


In [14]:

# Defining emotion labels
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Function to preprocess a single image for prediction
def ef(image):
    img = load_img(image, color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [15]:

# Example prediction
image = 'images/train/sad/42.jpg'
print("Original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("Model prediction is", pred_label)


Original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Model prediction is fear
